In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


folder1_path = '/content/gdrive/MyDrive'
folder2_path = '/content/gdrive/MyDrive/train_img'
folder3_path = '/content/gdrive/Othercomputers/내 노트북/test_img'

Mounted at /content/gdrive


In [ ]:
!pip install segmentation_models_pytorch --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.4 MB/s eta 0:00:00


In [ ]:
import segmentation_models_pytorch as smp


## Decode mask

In [ ]:
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

## Validation offline augmentation
crop with (256 * 256)

## Train Model

#### Dataset

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from typing import List
import torch.nn.functional as F
import sys, os
import os.path as osp
#sys.path.append(osp.dirname(osp.dirname(__file__)))

import os
import cv2
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class SatelliteDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file) # 데이터 로드
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 1] #self.data의 idx번째 행의 1번 열에 위치한 이미지 경로
        if './train_img' in img_path:
          new_path = img_path.replace('./train_img', '/content/gdrive/MyDrive/train_img')
        elif './test_img' in img_path:
          new_path = img_path.replace('./test_img', '/content/gdrive/Othercomputers/내 노트북/test_img')

        image = cv2.imread(new_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # 이미지는 OpenCV를 사용하여 로드하고 RGB로 변환.
        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image # 추론 모드 시 이미지만 반환

        mask_rle = self.data.iloc[idx, 2] # self.data의 idx번째 행의 2번 열에 위치한 RLE(Run-Length Encoding)로 인코딩된 마스크 정보를 가져옵니다.
        mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))
        # 훈련 모드 시 rle 문자열로 인코딩된 마스크를 디코딩, 변환하여 이미지와 마스크를 반환.
        # RLE로 인코딩된 마스크 정보를 디코딩하여 원래 형태의 마스크로 변환합니다. 변환된 마스크의 크기는 이미지의 높이와 너비와 동일합니다.
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

In [ ]:
train_transform = A.Compose(
            [
                A.OneOf(
                    [
                        A.RandomBrightness(p=1),
                        A.RandomBrightnessContrast(p=1),
                        A.Emboss(p=1),
                        A.RandomShadow(p=1),
                        A.Cutout(num_holes=8, max_h_size=32, max_w_size=32, fill_value=0, p=0.5),
                        A.NoOp(),
                    ],
                    p=1,
                ),
                A.OneOf(
                    [
                        A.Blur(p=1),
                        A.AdvancedBlur(p=1),
                        A.MotionBlur(p=1),
                    ],
                    p=0.6,
                ),
                A.OneOf(
                    [
                        A.NoOp(),
                        A.HorizontalFlip(p=0.5),
                        A.VerticalFlip(p=0.5),
                        A.ShiftScaleRotate(p=0.5),
                        A.Rotate(limit=90, p=1, border_mode=cv2.BORDER_REPLICATE),
                        A.RandomRotate90(p=1)
                    ],
                    p=1,
                ),
                A.RandomCrop(224, 224),
                A.Normalize(),
                ToTensorV2(transpose_mask=True)
            ]
        )


test_transform = A.Compose(
    [
        A.Resize(224, 224),
        A.Normalize(),
        ToTensorV2()
    ]
)

dataset = SatelliteDataset(csv_file=folder2_path+'/train.csv', transform=train_transform)
dataloader = DataLoader(dataset, batch_size=20, shuffle=True, num_workers=8)

/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:1613: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/dropout/cutout.py:50: FutureWarning: Cutout has been deprecated. Please use CoarseDropout
  warnings.warn(


#### Model

In [ ]:
class SMP(nn.Module):
    def __init__(self, encoder_name="resnet34", encoder_weights="imagenet", in_channels=3, classes=1):
        super().__init__()

        self.model = smp.UnetPlusPlus(
            encoder_name=encoder_name,
            encoder_weights=encoder_weights,
            in_channels=in_channels,
            classes=classes,
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
configs = {
    "model": {
        "encoder_name": "timm-regnety_320",
        "encoder_weights": "imagenet",
        "in_channels": 3,
        "classes": 1
    },
    "data": {
        "root": ".",
        "batch_size": 64
    },
}

#### Training

In [ ]:
model = SMP(
            encoder_name=configs["model"]["encoder_name"],
            encoder_weights=configs["model"]["encoder_weights"],
            in_channels=configs["model"]["in_channels"],
            classes=configs["model"]["classes"],
        )
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-regnet/regnety_320-ba464b29.pth" to /root/.cache/torch/hub/checkpoints/regnety_320-ba464b29.pth
100%|██████████| 554M/554M [00:34<00:00, 16.8MB/s]


In [ ]:
weight_file_path = '/content/gdrive/MyDrive/unet3_ag/sc_paper2_50.pth'
model.load_state_dict(torch.load(weight_file_path))

<All keys matched successfully>

In [ ]:
# 모델 초기화
# GPU를 사용할 경우

logs = []
# loss function과 optimizer 정의
#criterion = HybridLoss()
#criterion = DiceLoss()
criterion = torch.nn.BCEWithLogitsLoss() # 이진 교차 엔트로피
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

torch.autograd.set_detect_anomaly(True)


for epoch in range(20):  # 10 에폭 동안 학습합니다.
    model.train()
    epoch_loss = 0

    for images, masks in tqdm(dataloader):
        images = images.float().to(device)
        masks = masks.float().to(device)

        optimizer.zero_grad()
        out = model(images)

        loss = criterion(out, masks.unsqueeze(1))

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f'Epoch {epoch+50+1}, Loss: {epoch_loss/len(dataloader)}')
    log_epoch = {'epoch': epoch+1,'train_loss': epoch_loss}
    logs.append(log_epoch)
    scheduler.step(epoch_loss)  # 학습률 스케쥴러 호출

    if (10+epoch+1) % 5 == 0:
        torch.save(model.state_dict(), '/content/gdrive/MyDrive/unet3_ag/sc_paper2_' + str(50+epoch+1) + '.pth')


100%|██████████| 357/357 [13:16<00:00,  2.23s/it]


Epoch 51, Loss: 0.05659982593370085


100%|██████████| 357/357 [13:02<00:00,  2.19s/it]


Epoch 52, Loss: 0.056036042607798965


100%|██████████| 357/357 [13:01<00:00,  2.19s/it]


...

Epoch 59, Loss: 0.05667237515355192


100%|██████████| 357/357 [13:01<00:00,  2.19s/it]


Epoch 60, Loss: 0.052972411882935784


  2%|▏         | 6/357 [00:18<17:57,  3.07s/it]


KeyboardInterrupt: ignored

In [ ]:
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold
from tqdm import tqdm
from torchvision.transforms import Compose, ToTensor, Normalize
from torch.optim.lr_scheduler import ReduceLROnPlateau
from timm import create_model
from torch.utils.data import Subset

# Define your three models
import timm

def create_efficientnet_model(model_name, num_classes, pretrained=True):
    model = timm.create_model(model_name, pretrained=pretrained)
    num_ftrs = model.classifier.in_features
    model.classifier = nn.Linear(num_ftrs, num_classes)
    return model

model1 = create_efficientnet_model('tf_efficientnet_b4_ns', num_classes=1)

import timm

def create_vit_model(model_name, num_classes, pretrained=True):
    model = timm.create_model(model_name, pretrained=pretrained)
    num_ftrs = model.head.in_features
    model.head = nn.Linear(num_ftrs, num_classes)
    return model

model2 = create_vit_model('vit_base_patch16_224', num_classes=1)



# Move models to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1 = model1.to(device)
model2 = model2.to(device)

# Define your custom dataset and dataloader
# your_dataset = ...
# dataloader = DataLoader(your_dataset, batch_size=64, shuffle=True)

# Define the loss function and optimizer
criterion = torch.nn.BCEWithLogitsLoss()  # Binary cross-entropy
optimizer1 = torch.optim.Adam(model1.parameters(), lr=0.001)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.001)

scheduler1 = ReduceLROnPlateau(optimizer1, mode='min', factor=0.1, patience=5, verbose=True)
scheduler2 = ReduceLROnPlateau(optimizer2, mode='min', factor=0.1, patience=5, verbose=True)

# Define your k-fold parameters
k = 5  # number of folds
num_epochs = 10

# Split your dataset into k folds using KFold
kf = KFold(n_splits=k, shuffle=True)

# Training loop for k-fold ensemble
for fold, (train_index, _) in enumerate(kf.split(dataset)):
    train_dataset = Subset(dataset, train_index)
    train_dataloader = DataLoader(train_dataset, batch_size=12, shuffle=True, num_workers=8)

    print(f"Fold {fold + 1}/{k}")

    for epoch in range(num_epochs):
        model1.train()
        model2.train()
        model3.train()
        epoch_loss1 = 0
        epoch_loss2 = 0
        epoch_loss3 = 0

        for images, masks in tqdm(train_dataloader):
            images = images.float().to(device)
            masks = masks.float().to(device)

            # Model 1
            optimizer1.zero_grad()
            out1 = model1(images)
            loss1 = criterion(out1, masks.unsqueeze(1))
            loss1.backward()
            optimizer1.step()
            epoch_loss1 += loss1.item()

            # Model 2
            optimizer2.zero_grad()
            out2 = model2(images)
            loss2 = criterion(out2, masks.unsqueeze(1))
            loss2.backward()
            optimizer2.step()
            epoch_loss2 += loss2.item()

        print(f'Epoch {epoch + 1}, Model 1 Loss: {epoch_loss1 / len(train_dataloader)}, '
              f'Model 2 Loss: {epoch_loss2 / len(train_dataloader)}, ')

        scheduler1.step(epoch_loss1)  # Learning rate scheduler for model 1
        scheduler2.step(epoch_loss2)  # Learning rate scheduler for model 2

        if (epoch+1) % 2 == 0:
              torch.save(model1.state_dict(), '/content/gdrive/MyDrive/unet3_ag/en_kf1' +str(fold)+ '_' + str(epoch+1) + '.pth')
              torch.save(model2.state_dict(), '/content/gdrive/MyDrive/unet3_ag/en_kf2' +str(fold)+ '_' + str(epoch+1) + '.pth')


        # Validation and early stopping (you can add your early stopping logic here)

# After training all folds, you can use the ensemble of the three models for prediction


Fold 1/5


100%|██████████| 476/476 [17:30<00:00,  2.21s/it]


Epoch 1, Model 1 Loss: 0.1365374520248851, Model 2 Loss: 0.15473831480196795, Model 3 Loss: 0.16256318732845684


100%|██████████| 476/476 [17:27<00:00,  2.20s/it]


Epoch 2, Model 1 Loss: 0.10424827029850303, Model 2 Loss: 0.11239131988503602, Model 3 Loss: 0.134325081983409


...

Epoch 8, Model 1 Loss: 0.049874180538517214, Model 2 Loss: 0.05756206009458719, Model 3 Loss: 0.06716413397592406


100%|██████████| 476/476 [17:17<00:00,  2.18s/it]


Epoch 9, Model 1 Loss: 0.052475406807855146, Model 2 Loss: 0.0603634535194132, Model 3 Loss: 0.06973228339465852


100%|██████████| 476/476 [17:17<00:00,  2.18s/it]


Epoch 10, Model 1 Loss: 0.05104868010688229, Model 2 Loss: 0.05807658687171315, Model 3 Loss: 0.06809380329253167


In [ ]:
# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

test_dataset = SatelliteDataset('/content/gdrive/MyDrive/test.csv', transform=test_transform, infer=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

with torch.no_grad():
    model.eval()
    result = []
    for images in tqdm(test_dataloader):
        images = images.float().to(device)

        outputs = model(images)
        masks = torch.sigmoid(outputs).cpu().numpy()
        masks = np.squeeze(masks, axis=1)
        masks = (masks > 0.35).astype(np.uint8) # Threshold = 0.35

        for i in range(len(images)):
            mask_rle = rle_encode(masks[i])
            if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
                result.append(-1)
            else:
                result.append(mask_rle)

submit = pd.read_csv(folder2_path+'/sample_submission.csv')
submit['mask_rle'] = result
submit.to_csv(folder2_path+'/sch_new_paper2_50.csv', index=False)

100%|██████████| 3790/3790 [3:08:03<00:00,  2.98s/it]


In [ ]:

# Define your three models
model1 = SMP(
            encoder_name="timm-regnety_320",
            encoder_weights="imagenet",
            in_channels=3,
            classes=1,
        )

model2 = SMP(
            encoder_name="resnet152",
            encoder_weights="imagenet",
            in_channels=3,
            classes=1,
        )

model3 = smp.DeepLabV3Plus(encoder_name="resnet101", encoder_weights="imagenet", in_channels=3, classes=1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1 = model1.to(device)
model2 = model2.to(device)
model3 = model3.to(device)


weight_file_path = '/content/gdrive/MyDrive/unet3_ag/en_kf14_10.pth'
model1.load_state_dict(torch.load(weight_file_path))

weight_file_path = '/content/gdrive/MyDrive/unet3_ag/en_kf24_10.pth'
model2.load_state_dict(torch.load(weight_file_path))

weight_file_path = '/content/gdrive/MyDrive/unet3_ag/en_kf34_10.pth'
model3.load_state_dict(torch.load(weight_file_path))

def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
import torch
from torchvision.transforms import Compose, ToTensor, Normalize
from torch.utils.data import DataLoader
from timm import create_model
from tqdm import tqdm

# Define your three models (model1, model2, and model3)
test_dataset = SatelliteDataset('/content/gdrive/MyDrive/test.csv', transform=test_transform, infer=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)


# Define your test dataset and dataloader
# test_dataset = ...
# test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Set models to evaluation mode
model1.eval()
model2.eval()
model3.eval()

predictions = []  # List to store the final ensemble predictions

with torch.no_grad():
    result = []
    for images in tqdm(test_dataloader):
        images = images.float().to(device)
        # Forward pass through all three models
        out1 = model1(images)
        out2 = model2(images)
        out3 = model3(images)

        # Combine the predictions from the three models (e.g., averaging)
        ensemble_output = (out1 + out2 + out3) / 3

        masks = torch.sigmoid(ensemble_output).cpu().numpy()
        masks = np.squeeze(masks, axis=1)
        masks = (masks > 0.35).astype(np.uint8) # Threshold = 0.35
        # Apply activation function (e.g., sigmoid) if required for binary segmentation
        # ensemble_output = torch.sigmoid(ensemble_output)

        for i in range(len(images)):
            mask_rle = rle_encode(masks[i])
            if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
                result.append(-1)
            else:
                result.append(mask_rle)


submit = pd.read_csv(folder2_path+'/sample_submission.csv')
submit['mask_rle'] = result
submit.to_csv(folder2_path+'/en_3.csv', index=False)


 98%|█████████▊| 3729/3790 [58:46<01:49,  1.80s/it]

In [ ]:
len(result)

16

In [ ]:
class DiceLoss(nn.Module):
    def __init__(self):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        inputs = F.sigmoid(inputs) # sigmoid를 통과한 출력이면 주석처리

        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice = (2.*intersection + smooth) / (inputs.sum() + targets.sum() + smooth)

        return 1 - dice


In [ ]:
class MyEnsemble(nn.Module):
  def __init__(self, modelA, modelB, modelC, output):
    super(MyEnsemble, self).__init__()
    self.modelA = modelA
    self.modelB = modelB
    self.modelC = modelC

    self.fc1 = nn.Linear(num_classes, output)

  def forward(self,x):
    out1 = self.modelA(x)
    out2 = self.modelB(x)
    out3 = self.modelC(x)

    out = out1 + out2 + out3
    x = torch.softmax(x, dim=1)
    return x

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import torchvision.models as models

densenet161 = models.densenet161(pretrained=True).to(device)
resnet101 = models.resnet101(pretrained=True).to(device)
googlenet = models.googlenet(pretrained=True).to(device)

densenet161.classifier = nn.Linear(in_features=densenet161.classifier.in_features, out_features=num_classes)
resnet101.fc = nn.Linear(in_features=resnet101.fc.in_features, out_features=num_classes)
googlenet.fc = nn.Linear(in_features=googlenet.fc.in_features, out_features=num_classes)

densenet161 = densenet161.to(device)
resnet101 = resnet101.to(device)
googlenet = googlenet.to(device)



densenet161, history, best_epoch = train_and_val(densenet161, criterion, optimizer, dataloader['train'], dataloader['valid'], 2)
densenet = [densenet161, history, best_epoch]
resnet101, history, best_epoch = train_and_val(resnet101, criterion, optimizer, dataloader['train'], dataloader['valid'], 2)
resnet = [resnet101, history, best_epoch]
googlenet, history, best_epoch = train_and_val(googlenet, criterion, optimizer, dataloader['train'], dataloader['valid'], 2)
googlenet = [googlenet, history, best_epoch]

ensemble = MyEnsemble(densenet[1], resnet[1], googlenet[1], 10)